# The Battle of Neighborhoods (Final)

## Introduction/Business Problem

### Background

An international renowned, multi-Michelin starred chef Gordon Ramsey has opened a string of award-winning restaurants across the globe and now he is ready to open his new italian restaurant in Toronto. Looking for a perfect location for his new restaurant in Toronto, Gordon Ramsey asked me to use my data science knowledge to provide valuable insights and recommendation for his new restaurant. <br>
He specifically wants to know:<br>
* Floating population in Toronto
* Restaurants that could be competitors to his new restaurant
* Whether there are markets or farms near the location for affordable, quick and fresh ingredients
* Average income of residents in each neighborhood (because most of his foods are expensive)
* Characteristics of residents in each neighborhood (race, dining culture, etc...)
* Whether there's school/university in each neighborhood 

### Business Problem

The challenge is finding a best location for Ramsey's new italian restaurent in Toronto. Such location should have high demand for Ramsey's luxurious dishes. In addition, it should be close enough to markets/farms for fresh and quick ingredients distribution to his restaurant. 

### Target Audience

Beside my client Gordon Ramsey, the target audience would be the stakeholders and employees of Ramsey's food company. Anyone who are interested in food business also would be the target audience.  

## Question #2

## Data Description

I will be using the data below in order to solve the business problem.
* Boroughs and neighborhood in Toronto including its latitude and longitude
* Floating population data in each borough
* Location of markets/farms
* Average income of residents in each neighborhood
* Location of Universities 
* Proportion of race in each neighborhood

### How to use data to solve the problem

I will collect data as follows:
* Foursquare and geopy will be used to retreive geodata (latitude and longitude)
* Address and geodata for markets/farms will be achieved by geopy and Nominatim
* BeautifulSoup will be utilized for retriving data from URL (for floating population and avg income data)
* Location of universities and proportion of race will be read through CSV file.
* Use Folium for drawing map for Toronto and neighborhood

#### Successful collection of the data will allow us to answer the following questions:
* Which location has high floating population?
* What would be the possible candidates in terms of neighborhood?
* What is the characteristics of residents in each neighborhood?
* What is the distance between our possible candidates and markets/farms for each neighborhood?
* Who is the possible competitor?

### Import Libraries

In [2]:
import pandas as pd
import numpy as np
import requests # handle request
from bs4 import BeautifulSoup # use for getting data from URL

import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

print("Libraries imported Success!")

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          63 KB

The following packages will be UPDATED:

  geopy                                 1.22.0-pyh9f0ad1d_0 --> 2.0.0-pyh9f0ad1d_0



geopy-2.0.0          | 63 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: / WARNING conda.core.path_actions:verify(963): Unable to create environments file. Path not writable.
  environment location: /Users/Macintosh/.conda/environments.txt

done
Executing transaction: done
Solving environment: done

# All requested packages already i

### Load data from URL

In [3]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
req = requests.get(URL)

soup = BeautifulSoup(req.content, "html5lib")
table = soup.find('div', {'id':'container'})
# print(soup.prettify())

### Prepare Dataset

In [4]:
lst = []
n = 0

all_tds = soup.find_all("td")               # finds all things that starts with <td> and ends with </td>
all_tds_lst = [x.get_text().strip("\n") for x in all_tds]  # convert bs4 list into a list of strings
len_tds = [a for a in range(0, 3)]                      # create list of 0, 1, 2s

for element in all_tds_lst:
    if element == "": 
        break # control for things that goes beyond our last </td>
    if n == 0:
        lst.append([])
        lst[-1].append(element)
        n += 1
    elif n == 2:
        lst[-1].append(element)
        n = 0 # reset n 
    else:
        lst[-1].append(element)
        n += 1

In [5]:
# create pandas DF
df = pd.DataFrame(data = lst, columns = ['PostalCode', 'Borough', 'Neighborhood'])
#df = df[~df.Borough.str.contains("Not assigned")]
df = df[~df.Neighborhood.str.contains("Not assigned")]
df = df.reset_index()
df = df.drop(columns='index', axis=1)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [6]:
df.shape

(103, 3)

#### Get latitude and longitude for each borough

In [7]:
df_latlng = pd.read_csv("http://cocl.us/Geospatial_data")
df_latlng.rename(columns={'Postal Code':'PostalCode'}, inplace=True)
df_latlng.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
toronto_df = pd.merge(df, df_latlng, on='PostalCode')
toronto_df = toronto_df.drop_duplicates(subset=['Borough', 'Neighborhood'], keep=False)
toronto_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


#### Get geodata for Toronto by geopy, Nominatim

In [9]:
address = 'Toronto, ON, Canada'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Map of Toronto and mark circle for each borough using Folium

In [10]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Collect demographic data of Toronto neighborhood from URL

In [11]:
demogrpah_URL = 'https://en.wikipedia.org/wiki/Demographics_of_Toronto_neighbourhoods'
r = requests.get(demogrpah_URL)

soup = BeautifulSoup(r.content, "html5lib")
table = soup.find('div', {'id':'container'})
#print(soup.prettify())

In [12]:
lst_2 = []
n = 0

all_tds = soup.find_all("td")   
all_tds_lst = [x.get_text().strip('\n') for x in all_tds] 
# retrieve the table data that we are interested in
start_idx = all_tds_lst.index('Agincourt')
end_idx = all_tds_lst.index('General outline\nDemographics\nName\nFlag\nCoat of arms\nSister cities\nNotable Torontonians')
demographic_lst = all_tds_lst[start_idx:end_idx]
demographic_lst = [x for x in demographic_lst if x != '']

# separate into 12 columns
for element in demographic_lst:
    if n == 0:
        lst_2.append([])
        lst_2[-1].append(element)
        n += 1
    elif n == 11:
        lst_2[-1].append(element)
        n = 0 # reset n 
    else:
        lst_2[-1].append(element)
        n += 1
#print(lst_2)

In [13]:
# Convert lst into DataFrame
cols = ['Neighborhood', 'Borough', 'CensusTracts', 'Population', 
        'LandArea', 'Density (people/km2)', 'PopulationChange %', 'AvgIncome',
       'TransitCommuting %', 'Renters %', '2nd common language (name)', '2nd common language %']
demograph_df_all = pd.DataFrame(data = lst_2, columns=cols)
demograph_df_all

,Neighborhood,Borough,CensusTracts,Population,LandArea,Density (people/km2),PopulationChange %,AvgIncome,TransitCommuting %,Renters %,2nd common language (name),2nd common language %
0,Agincourt,S,"0377.01, 0377.02, 0377.03, 0377.04, 0378.02, 0...","44,577",12.45,3580,4.6,"25,750",11.1,5.9,Cantonese (19.3%),19.3% Cantonese
1,Alderwood,E,"0211.00, 0212.00","11,656",4.94,2360,-4.0,"35,239",8.8,8.5,Polish (6.2%),06.2% Polish
2,Alexandra Park,OCoT,0039.00,"4,355",0.32,"13,609",0.0,"19,687",13.8,28.0,Cantonese (17.9%),17.9% Cantonese
3,Allenby,OCoT,0140.00,"2,513",0.58,4333,-1.0,"245,592",5.2,3.4,Russian (1.4%),01.4% Russian
4,Amesbury,NY,"0280.00, 0281.01, 0281.02","17,318",3.51,"4,934",1.1,"27,546",16.4,19.7,Spanish (6.1%),06.1% Spanish
...,...,...,...,...,...,...,...,...,...,...,...,...
169,Woburn,S,"0356.00, 0357.01, 0357.02, 0363.07, 0364.01, 0...","48,507",13.34,3636,-1.5,"26,190",13.3,16.0,Gujarati (9.1%),09.1% Gujarati
170,Wychwood,OCoT,0116.00,"4,182",0.68,6150,-2.0,"53,613",17.1,20.1,Portuguese (2.7%),02.7% Portuguese
171,York Mills,NY,"0273.01, 0273.02, 0274.01, 0274.02","17,564",7.29,2409,2.0,"92,099",10.0,11.8,Korean (4.0%),04.0% Korean
172,York University Heights,NY,"0311.02, 0311.03, 0311.04, 0311.05, 0311.06","26,140",13.21,1979,-1.2,"24,432",15.2,20.4,Italian (6.6%),06.6% Italian


In [14]:
# filter columns that we are interested in
demo_df = demograph_df_all.drop(['CensusTracts', 'Borough', 'LandArea', 'TransitCommuting %', 'Renters %', 
                   '2nd common language (name)', '2nd common language %'], axis=1)
demo_df

,Neighborhood,Population,Density (people/km2),PopulationChange %,AvgIncome
0,Agincourt,"44,577",3580,4.6,"25,750"
1,Alderwood,"11,656",2360,-4.0,"35,239"
2,Alexandra Park,"4,355","13,609",0.0,"19,687"
3,Allenby,"2,513",4333,-1.0,"245,592"
4,Amesbury,"17,318","4,934",1.1,"27,546"
...,...,...,...,...,...
169,Woburn,"48,507",3636,-1.5,"26,190"
170,Wychwood,"4,182",6150,-2.0,"53,613"
171,York Mills,"17,564",2409,2.0,"92,099"
172,York University Heights,"26,140",1979,-1.2,"24,432"


### Get Latitude and Longitude for each neighborhoods

In [15]:
lat_lng_list = []
not_avail = [] # There are several neighborhoods that are not available in geopy geolocator

geolocator = Nominatim(user_agent='myGeocoder')
for neighborhood in demo_df['Neighborhood']:
    lat_lng_list.append([])
    city ="Toronto"
    country ="Canada"
    location = geolocator.geocode(neighborhood+','+city+','+ country)
    try:
        lat_lng_list[-1].append(location.latitude)
        lat_lng_list[-1].append(location.longitude)
        #print('{}: Latitude = {}, Longitude = {}'.format(neighborhood, location.latitude, location.longitude))
    except:
        not_avail.append(neighborhood)

# print(lat_lng_list)
# print(not_avail)

In [22]:
# drop the neighbordhoods that are not available from demo_df
for s in not_avail:
    demo_df = demo_df[demo_df['Neighborhood'] != s]

# create new DF for lat lng for each neighborhood
lat_list = [lat_lng_list[x][0] for x in range(len(lat_lng_list)) if len(lat_lng_list[x]) != 0]
lng_list = [lat_lng_list[x][1] for x in range(len(lat_lng_list)) if len(lat_lng_list[x]) != 0]
lat_lng_df = pd.DataFrame({'Neighborhood':demo_df['Neighborhood'],
                           'Latitude':lat_list,
                          'Longitude':lng_list})
# now, merge lat_lng_df and demo_df by 'Neighborhood'
complete_tor_df = pd.merge(lat_lng_df, demo_df, on='Neighborhood')
complete_tor_df

,Neighborhood,Latitude,Longitude,Population,Density (people/km2),PopulationChange %,AvgIncome
0,Agincourt,43.785353,-79.278549,"44,577",3580,4.6,"25,750"
1,Alderwood,43.601717,-79.545232,"11,656",2360,-4.0,"35,239"
2,Alexandra Park,43.650787,-79.404318,"4,355","13,609",0.0,"19,687"
3,Allenby,43.711351,-79.553424,"2,513",4333,-1.0,"245,592"
4,Amesbury,43.706162,-79.483492,"17,318","4,934",1.1,"27,546"
...,...,...,...,...,...,...,...
159,Woburn,43.759824,-79.225291,"48,507",3636,-1.5,"26,190"
160,Wychwood,43.682122,-79.423839,"4,182",6150,-2.0,"53,613"
161,York Mills,43.744039,-79.406657,"17,564",2409,2.0,"92,099"
162,York University Heights,43.758781,-79.519434,"26,140",1979,-1.2,"24,432"


### Access to Foursquare to top venues near each neighborhood

In [23]:
CLIENT_ID = 'HT110NTAISUECGEVJNJ0ER0XPBTJGEUN5RTHQZX5TBI2JHHZ' # your Foursquare ID
CLIENT_SECRET = '5R332C1DX3M5FHJEPJHRVGMA1A41FJ01FJY13450I4OBAHKO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HT110NTAISUECGEVJNJ0ER0XPBTJGEUN5RTHQZX5TBI2JHHZ
CLIENT_SECRET:5R332C1DX3M5FHJEPJHRVGMA1A41FJ01FJY13450I4OBAHKO


In [24]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [25]:
# # Clean the json and structure it into a pandas dataframe
# venues = results['response']['groups'][0]['items']
    
# nearby_venues = json_normalize(venues) # flatten JSON

# # filter columns
# filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
# nearby_venues =nearby_venues.loc[:, filtered_columns]

# # filter the category for each row
# nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# # clean columns
# nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

# nearby_venues.head()

In [26]:
def getVenues (name, latitude, longitude, rad=500, lim=100):
    venues_list=[]
    for name, lat, lng in zip(name, latitude, longitude):   
        # request data from Foursquare
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        lat, 
        lng, 
        rad, 
        lim)
        
        # make the GET request
        results = requests.get(url)#.json()["response"]['groups'][0]['items']
        results = results.json()["response"]

        try:
            results = results['groups'][0]['items']
        except:
            continue
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
venues_df = getVenues(complete_tor_df['Neighborhood'], complete_tor_df['Latitude'], complete_tor_df['Longitude'])

In [28]:
venues_df

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agincourt,43.785353,-79.278549,Tim Hortons,43.785637,-79.279215,Coffee Shop
1,Agincourt,43.785353,-79.278549,One2 Snacks,43.787048,-79.276658,Asian Restaurant
2,Agincourt,43.785353,-79.278549,In Cheon House Korean & Japanese Restaurant 인천관,43.786468,-79.275693,Korean Restaurant
3,Agincourt,43.785353,-79.278549,Beef Noodle Restaurant 老李牛肉麵,43.785937,-79.276031,Chinese Restaurant
4,Agincourt,43.785353,-79.278549,Congee King,43.785908,-79.276042,Chinese Restaurant
...,...,...,...,...,...,...,...
3597,Yorkville,43.671386,-79.390168,The Yorkville Club,43.671636,-79.395015,Gym
3598,Yorkville,43.671386,-79.390168,Sofia,43.670592,-79.391849,Italian Restaurant
3599,Yorkville,43.671386,-79.390168,Yorkville Village,43.671096,-79.394417,Shopping Mall
3600,Yorkville,43.671386,-79.390168,The One Eighty,43.668575,-79.388210,American Restaurant


In [29]:
# check how many venues were returned for each neighborhood
venues_df.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,13,13,13,13,13,13
Alderwood,9,9,9,9,9,9
Alexandra Park,100,100,100,100,100,100
Allenby,8,8,8,8,8,8
Amesbury,6,6,6,6,6,6
...,...,...,...,...,...,...
Woburn,23,23,23,23,23,23
Wychwood,52,52,52,52,52,52
York Mills,20,20,20,20,20,20


In [30]:
# find out how many unique categories can be curated from all the returned venues
print('There are {} uniques categories.'.format(len(venues_df['Venue Category'].unique())))

There are 316 uniques categories.


### Analyze Each Neighborhood

In [31]:
# one hot encoding (convert venue category variables into binary variable)
venues_onehot = pd.get_dummies(venues_df[['Venue Category']], prefix='', prefix_sep='')

# add neighborhood column back to dataframe
venues_onehot['Neighborhood'] = venues_df['Neighborhood'] 

# find the idx of 'Neighborhood' column
a = list(venues_onehot.columns)
a.index('Neighborhood') #212

# now, move the Neighborhood column to 1st
fixed_columns = [venues_onehot.columns[212]] + list(venues_onehot.columns[:212]) + list(venues_onehot.columns[213:])
venues_onehot = venues_onehot[fixed_columns]
venues_onehot.head()

,Nightclub,ATM,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
venues_onehot.shape

(3602, 316)

In [33]:
venues_grouped = venues_onehot.groupby('Neighborhood').mean().reset_index()
venues_grouped.head()

,Neighborhood,Nightclub,ATM,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Animal Shelter,Antique Shop,Aquarium,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.00,0.0,0.0,0.0,0.000,0.00,0.0,0.0,0.0,...,0.0,0.0,0.076923,0.0,0.0,0.00,0.0,0.0,0.0,0.00
1,Alderwood,0.00,0.0,0.0,0.0,0.000,0.00,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.00
2,Alexandra Park,0.01,0.0,0.0,0.0,0.000,0.01,0.0,0.0,0.0,...,0.0,0.0,0.010000,0.0,0.0,0.01,0.0,0.0,0.0,0.01
3,Allenby,0.00,0.0,0.0,0.0,0.125,0.00,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.00
4,Amesbury,0.00,0.0,0.0,0.0,0.000,0.00,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.0,0.00


In [34]:
venues_grouped.shape

(158, 316)

In [35]:
num_top_venues = 5

for hood in venues_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = venues_grouped[venues_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                  venue  freq
0    Chinese Restaurant  0.15
1         Train Station  0.08
2            Food Court  0.08
3                Bakery  0.08
4  Cantonese Restaurant  0.08


----Alderwood----
            venue  freq
0     Pizza Place  0.22
1  Sandwich Place  0.11
2             Pub  0.11
3             Gym  0.11
4    Dance Studio  0.11


----Alexandra Park----
                    venue  freq
0                     Bar  0.10
1    Caribbean Restaurant  0.05
2                    Café  0.04
3  Furniture / Home Store  0.04
4        Asian Restaurant  0.02


----Allenby----
                venue  freq
0           Bookstore  0.12
1   Fish & Chips Shop  0.12
2  African Restaurant  0.12
3                Café  0.12
4        Intersection  0.12


----Amesbury----
                venue  freq
0         Coffee Shop  0.17
1        Intersection  0.17
2                Park  0.17
3  Athletics & Sports  0.17
4         Gas Station  0.17


----Armour Heights----
           venue  freq


### Let's put that into a pandas dataframe
#### First, let's write a function to sort the venues in descending order.

In [36]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns list according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = venues_grouped['Neighborhood']

# now fill the data of the new data frame
for ind in np.arange(venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Hong Kong Restaurant,Train Station,Coffee Shop,Rental Car Location,Cantonese Restaurant,Shopping Mall,Korean Restaurant,Bakery,Asian Restaurant
1,Alderwood,Pizza Place,Coffee Shop,Skating Rink,Dance Studio,Gym,Pub,Sandwich Place,Pharmacy,Falafel Restaurant,Farmers Market
2,Alexandra Park,Bar,Caribbean Restaurant,Furniture / Home Store,Café,Gym / Fitness Center,Park,Dessert Shop,Coffee Shop,Liquor Store,Mediterranean Restaurant
3,Allenby,Restaurant,Bookstore,Café,African Restaurant,Big Box Store,Fish & Chips Shop,Fast Food Restaurant,Intersection,Yoga Studio,Dumpling Restaurant
4,Amesbury,Intersection,Bank,Athletics & Sports,Coffee Shop,Gas Station,Park,Event Space,Donut Shop,Dumpling Restaurant,Eastern European Restaurant


## Cluster Neighborhoods
#### Run k-means to cluster the neighborhood into 5 clusters

In [38]:
# set number of clusters
kclusters = 5

venues_grouped_clustering = venues_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(venues_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 1, 1, 1, 1, 1, 3, 0, 1, 1], dtype=int32)

In [39]:
# add clustering labels
#neighborhoods_venues_sorted.drop('Cluster Labels', axis=1, inplace=True)
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = complete_tor_df
complete_tor_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

# drop the neighborhoods that contains NaN
toronto_merged.dropna(subset=['Cluster Labels'], axis = 0, inplace=True)
toronto_merged.reset_index(inplace=True)
toronto_merged # check the last columns!

,index,Neighborhood,Latitude,Longitude,Population,Density (people/km2),PopulationChange %,AvgIncome,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Agincourt,43.785353,-79.278549,"44,577",3580,4.6,"25,750",1.0,Chinese Restaurant,Hong Kong Restaurant,Train Station,Coffee Shop,Rental Car Location,Cantonese Restaurant,Shopping Mall,Korean Restaurant,Bakery,Asian Restaurant
1,1,Alderwood,43.601717,-79.545232,"11,656",2360,-4.0,"35,239",1.0,Pizza Place,Coffee Shop,Skating Rink,Dance Studio,Gym,Pub,Sandwich Place,Pharmacy,Falafel Restaurant,Farmers Market
2,2,Alexandra Park,43.650787,-79.404318,"4,355","13,609",0.0,"19,687",1.0,Bar,Caribbean Restaurant,Furniture / Home Store,Café,Gym / Fitness Center,Park,Dessert Shop,Coffee Shop,Liquor Store,Mediterranean Restaurant
3,3,Allenby,43.711351,-79.553424,"2,513",4333,-1.0,"245,592",1.0,Restaurant,Bookstore,Café,African Restaurant,Big Box Store,Fish & Chips Shop,Fast Food Restaurant,Intersection,Yoga Studio,Dumpling Restaurant
4,4,Amesbury,43.706162,-79.483492,"17,318","4,934",1.1,"27,546",1.0,Intersection,Bank,Athletics & Sports,Coffee Shop,Gas Station,Park,Event Space,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,159,Woburn,43.759824,-79.225291,"48,507",3636,-1.5,"26,190",1.0,Fast Food Restaurant,Coffee Shop,Discount Store,Bank,Luggage Store,Gym,Pharmacy,Paper / Office Supplies Store,Department Store,Sandwich Place
154,160,Wychwood,43.682122,-79.423839,"4,182",6150,-2.0,"53,613",1.0,Italian Restaurant,Ice Cream Shop,Restaurant,Coffee Shop,Café,Sushi Restaurant,Bakery,Indian Restaurant,Pizza Place,Dance Studio
155,161,York Mills,43.744039,-79.406657,"17,564",2409,2.0,"92,099",1.0,Coffee Shop,Gym,Gym / Fitness Center,Tennis Court,Thai Restaurant,Business Service,Optical Shop,Restaurant,Japanese Restaurant,Burrito Place
156,162,York University Heights,43.758781,-79.519434,"26,140",1979,-1.2,"24,432",1.0,Pizza Place,Grocery Store,Fast Food Restaurant,Discount Store,Pharmacy,Gym / Fitness Center,Falafel Restaurant,Fried Chicken Joint,Beer Store,Liquor Store


In [40]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    cluster = int(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters 
#### Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster

#### Cluster 0

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Density (people/km2),PopulationChange %,AvgIncome,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Bathurst Manor,3187,12.3,"34,169",0.0,Playground,Convenience Store,Baseball Field,Park,Event Space,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
15,Bracondale Hill,8618,-3.0,"41,605",0.0,Park,Bakery,Art Gallery,Bar,Coffee Shop,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
20,Carleton Village,8843,-4.0,"23,301",0.0,Deli / Bodega,Coffee Shop,Jewelry Store,Park,Dog Run,Fish & Chips Shop,Ethiopian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant
30,Crescent Town,"20,393",-10.0,"23,021",0.0,Golf Course,Convenience Store,Metro Station,Park,Ethiopian Restaurant,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
31,Davenport,8870,-6.9,"28,335",0.0,Dog Run,Convenience Store,Music Venue,Coffee Shop,Park,Ethiopian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
51,Forest Hill,5530,-0.2,"101,631",0.0,Playground,Bank,Arts & Crafts Store,Park,Ethiopian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
61,Henry Farm,3066,-6.0,"56,395",0.0,Restaurant,Tennis Court,Park,Yoga Studio,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
62,High Park North,"10,434",-1.6,"46,437",0.0,Park,Convenience Store,Tennis Court,Baseball Field,Gym / Fitness Center,Yoga Studio,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
66,Humber Summit,1618,2.1,"26,117",0.0,Construction & Landscaping,Bakery,Gym,Park,Ethiopian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
67,Humber Valley Village,2652,-0.1,"80,618",0.0,Park,Convenience Store,Bakery,Bus Stop,Skating Rink,Yoga Studio,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store


#### Cluster 1

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Density (people/km2),PopulationChange %,AvgIncome,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,3580,4.6,"25,750",1.0,Chinese Restaurant,Hong Kong Restaurant,Train Station,Coffee Shop,Rental Car Location,Cantonese Restaurant,Shopping Mall,Korean Restaurant,Bakery,Asian Restaurant
1,Alderwood,2360,-4.0,"35,239",1.0,Pizza Place,Coffee Shop,Skating Rink,Dance Studio,Gym,Pub,Sandwich Place,Pharmacy,Falafel Restaurant,Farmers Market
2,Alexandra Park,"13,609",0.0,"19,687",1.0,Bar,Caribbean Restaurant,Furniture / Home Store,Café,Gym / Fitness Center,Park,Dessert Shop,Coffee Shop,Liquor Store,Mediterranean Restaurant
3,Allenby,4333,-1.0,"245,592",1.0,Restaurant,Bookstore,Café,African Restaurant,Big Box Store,Fish & Chips Shop,Fast Food Restaurant,Intersection,Yoga Studio,Dumpling Restaurant
4,Amesbury,"4,934",1.1,"27,546",1.0,Intersection,Bank,Athletics & Sports,Coffee Shop,Gas Station,Park,Event Space,Donut Shop,Dumpling Restaurant,Eastern European Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
153,Woburn,3636,-1.5,"26,190",1.0,Fast Food Restaurant,Coffee Shop,Discount Store,Bank,Luggage Store,Gym,Pharmacy,Paper / Office Supplies Store,Department Store,Sandwich Place
154,Wychwood,6150,-2.0,"53,613",1.0,Italian Restaurant,Ice Cream Shop,Restaurant,Coffee Shop,Café,Sushi Restaurant,Bakery,Indian Restaurant,Pizza Place,Dance Studio
155,York Mills,2409,2.0,"92,099",1.0,Coffee Shop,Gym,Gym / Fitness Center,Tennis Court,Thai Restaurant,Business Service,Optical Shop,Restaurant,Japanese Restaurant,Burrito Place
156,York University Heights,1979,-1.2,"24,432",1.0,Pizza Place,Grocery Store,Fast Food Restaurant,Discount Store,Pharmacy,Gym / Fitness Center,Falafel Restaurant,Fried Chicken Joint,Beer Store,Liquor Store


#### Cluster 2

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Density (people/km2),PopulationChange %,AvgIncome,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
116,Rouge Hill,2878,0.9,"32,858",2.0,Train Station,Bus Line,Yoga Studio,Doctor's Office,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant


#### Cluster 3

In [44]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Density (people/km2),PopulationChange %,AvgIncome,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
6,Banbury,2442,5.0,"92,319",3.0,Park,Auto Garage,Tennis Court,Yoga Studio,Ethiopian Restaurant,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
22,Centennial,2544,0.5,"34,867",3.0,Bar,Park,Yoga Studio,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space
46,Eringate,3282,-3.4,"34,789",3.0,Park,Electronics Store,Yoga Studio,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Ethiopian Restaurant
54,Governor's Bridge/Bennington Heights,1129,4.0,"129,904",3.0,Park,Trail,Yoga Studio,Ethiopian Restaurant,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
73,Kingsview Village,4013,-6.2,"32,004",3.0,Ice Cream Shop,Park,Yoga Studio,Event Space,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
102,Old Mill/Baby Point,3748,1.0,"110,372",3.0,Park,River,Yoga Studio,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
106,Pelmo Park,2001,14.0,"32,002",3.0,Coffee Shop,Park,Yoga Studio,Event Space,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
109,Port Union,2310,-1.7,"48,117",3.0,Park,Yoga Studio,Dive Bar,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
115,Rouge,791,175.0,"29,230",3.0,Fast Food Restaurant,Park,Yoga Studio,Ethiopian Restaurant,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store


#### Cluster 4

In [45]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, 
                   toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighborhood,Density (people/km2),PopulationChange %,AvgIncome,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
75,Lansing,3285,35.0,"46,631",4.0,Playground,Yoga Studio,Event Space,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant
126,Steeles,5464,1.9,"26,660",4.0,Playground,Health & Beauty Service,Yoga Studio,Event Space,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store
128,Sunnylea,3366,-1.1,"51,398",4.0,Playground,Yoga Studio,Event Space,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant


## Methodology Section
#### Represent the main component of the report where you discuss and describe any exploratory data analysis that you did, any inferential statistical testing that you performed, if any, and what machine learnings were used and why.

### Which location has high floating population?

In [46]:
high_pop_df = pd.DataFrame({'Neighborhood':toronto_merged['Neighborhood'],
                            'Population':toronto_merged['Population'],
                            'Density (people/km2)':toronto_merged['Density (people/km2)']})

# first, change type of values in Population and Density
# str -> int (removing , btw nums)
int_lst = []
den_lst = []
for p, d in zip(high_pop_df['Population'], high_pop_df['Density (people/km2)']):
    p = int(p.replace(',', ''))
    d = int(d.replace(',', ''))
    int_lst.append(p)
    den_lst.append(d)
high_pop_df['Population'] = int_lst
high_pop_df['Density (people/km2)'] = den_lst

high_pop_df = high_pop_df.sort_values(by='Population', ascending=False).reset_index(drop=True)
high_pop_df.head(10)

,Neighborhood,Population,Density (people/km2)
0,Old East York,52220,6577
1,Woburn,48507,3636
2,Elia (Jane and Finch),48003,6267
3,Agincourt,44577,3580
4,Malvern,44324,5003
5,Willowdale,43144,5618
6,Downsview,36613,2270
7,Newtonbrook,36046,4110
8,Smithfield,34996,5426
9,Fairbank,34121,7720


In [47]:
high_density_df = high_pop_df.sort_values(by='Density (people/km2)', ascending=False).reset_index(drop=True)
high_density_df.head(10)

,Neighborhood,Population,Density (people/km2)
0,St. James Town,14666,63765
1,Bay Street Corridor,4787,43518
2,North York City Centre,10427,37239
3,Church and Wellesley,13397,24358
4,Crescent Town,8157,20393
5,Garden District,8240,15846
6,Parkdale,28367,13974
7,Alexandra Park,4355,13609
8,Niagara,6524,11862
9,Wallace Emerson,10338,11748


In [48]:
print('The neighborhood with highest floating population is {}, with {} people/km2'.format(
    high_density_df['Neighborhood'][0], high_density_df['Density (people/km2)'][0]))


The neighborhood with highest floating population is St. James Town, with 63765 people/km2


### Now that we found the neighborhoods with high floating population, we need to take into account for the characteristics of those neighborhoods. 

### Let's first consider avg income of each neighborhoods

In [49]:
# recall our demo_df
demo_df.head()

,Neighborhood,Population,Density (people/km2),PopulationChange %,AvgIncome
0,Agincourt,"44,577",3580,4.6,"25,750"
1,Alderwood,"11,656",2360,-4.0,"35,239"
2,Alexandra Park,"4,355","13,609",0.0,"19,687"
3,Allenby,"2,513",4333,-1.0,"245,592"
4,Amesbury,"17,318","4,934",1.1,"27,546"


In [50]:
# let's merge demo_df and high_density_df 
demo_df_modified = demo_df.drop(['Population', 'Density (people/km2)'], axis=1)

lst2 = []
for a in demo_df_modified['AvgIncome']:
    a = int(a.replace(',', ''))
    lst2.append(a)
demo_df_modified['AvgIncome'] = lst2

char_high_den_df = pd.merge(high_density_df, demo_df_modified, on='Neighborhood')
char_high_den_df.head(10)

,Neighborhood,Population,Density (people/km2),PopulationChange %,AvgIncome
0,St. James Town,14666,63765,-10.0,22341
1,Bay Street Corridor,4787,43518,3.0,40598
2,North York City Centre,10427,37239,33.0,34330
3,Church and Wellesley,13397,24358,8.8,37653
4,Crescent Town,8157,20393,-10.0,23021
5,Garden District,8240,15846,17.0,37614
6,Parkdale,28367,13974,-8.0,26314
7,Alexandra Park,4355,13609,0.0,19687
8,Niagara,6524,11862,-4.0,44611
9,Wallace Emerson,10338,11748,-1.8,25029


In [51]:
high_income = char_high_den_df.sort_values(by='AvgIncome', ascending=False).reset_index(drop=True)
high_income.head(10)

,Neighborhood,Population,Density (people/km2),PopulationChange %,AvgIncome
0,Allenby,2513,4333,-1.0,245592
1,Hoggs Hollow,3123,1132,2.0,222560
2,Lawrence Park,6653,1828,-5.1,214110
3,Rosedale,7672,2821,4.8,213941
4,Moore Park,4474,3959,-2.0,154825
5,Governor's Bridge/Bennington Heights,2112,1129,4.0,129904
6,Lytton Park,6494,5073,5.0,127356
7,South Hill,6218,4935,-3.8,120453
8,Armour Heights,4384,1914,2.0,116651
9,The Kingsway,8780,3403,7.9,110944


### The neighborhoods with high average income are most likely the neighborhoods with residents only. Therefore, we should focus on the population density of each neighborhoods. 
### In terms of floating population, these are the highest:
* St. James Town
* Bay Street Corridor
* North York City Centre
* Church and Wellesley
* Crescent Town

### Let's see the characteristics of these neighborhoods

* <b>St. James Town</b>
    * St. James Town is the largest high-rise community in Canada. It has been identified as one of 13 economically deprived neighbourhoods within the city. Due to its cultural and minority demographics, St. James Town is often thought as "the world within a block".
* <b>Bay Street Corridor</b>
    * "Bay Street" is frequently used as a metonym to refer to Toronto's Financial District and the Canadian financial sector as a whole, similar to Wall Street in the United States. 
* <b>North York City Centre</b>
    * North York City Centre became the largest of four central business districts in the new city outside Downtown Toronto
* <b>Church and Wellesley</b>
    * It is is an LGBT-oriented enclave with the core commercial strip located along Church Street from Wellesley south to Alexander.
* <b>Crescent Town</b>
    * It mainly consists of high-rise apartment complexes, built originally to take advantage of the opening of the adjacent Victoria Park subway station, which connects to the central quadrangle via a partially covered walkway.

### After observing the neighborhoods characteristics, Church and Wellesley and Crescent Town seem quite unsuitable for Ramsey's new restaurant. 
#### As we are seeking for the location with efficient circulation of population, Church and Wellesley has too strong LGBT culture for Ramsey's restaurant .
#### Since Crescent Town is mainly consisted of high-rise apartment complexes, it is also not suitable for Ramsey's new restaurant.

### Therefore, the possible candidates for Ramsey's new restaurant are:
* <b>St. James Town</b>
* <b>Bay Street Corridor</b>
* <b>North York City Centre</b>

## What is the distance between our possible candidates and markets/farms for each neighborhood?

### Since all the possible candidates are in Ontario, Toronto, we will focus on markets/farms in Ontario.

#### Bring the distributors in Ontario using BeautifulSoup

In [52]:
URL = 'http://betterfoodconcepts.ca/distributors/'
r = requests.get(URL)

soup = BeautifulSoup(r.content, "html5lib")
table = soup.find('div', {'id':'container'})
print(soup.prettify())

<!DOCTYPE html>
<html lang="en-US">
 <head>
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <link href="http://betterfoodconcepts.ca/xmlrpc.php" rel="pingback"/>
  <script type="text/javascript">
   document.documentElement.className = 'js';
  </script>
  <script>
   var et_site_url='http://betterfoodconcepts.ca';var et_post_id='12';function et_core_page_resource_fallback(a,b){"undefined"===typeof b&&(b=a.sheet.cssRules&&0===a.sheet.cssRules.length);b&&(a.onerror=null,a.onload=null,a.href?a.href=et_site_url+"/?et_core_page_resource="+a.id+et_post_id:a.src&&(a.src=et_site_url+"/?et_core_page_resource="+a.id+et_post_id))}
  </script>
  <title>
   Distributors | BetterFoodConcepts
  </title>
  <link href="//fonts.googleapis.com" rel="dns-prefetch"/>
  <link href="//s.w.org" rel="dns-prefetch"/>
  <link href="http://betterfoodconcepts.ca/feed/" rel="alternate" title="BetterFoodConcepts » Feed" type="application/rss+xml"/>
  <link href="http://betterfood

In [53]:
all_strongs = soup.find_all("p")
all_strong_lst = [x.get_text().strip('\n') for x in all_strongs] 

# retrieve the table data that we are interested in
start_idx = all_strong_lst.index('Macgregors Meat & Seafood Atlantic Grocery Distributors Centennial Chicago58 FKK Wholesale Cash & Carey Flanagan Foodservice GFSThe Butcher Shoppe')
end_idx = all_strong_lst.index('Stewarts Summit Foods Sysco Vending Products of Canada Wallace & Carrey Inc. Ward Foods Worldwide Specialty Foods')
food_lst = all_strong_lst[start_idx:end_idx+1]
food_lst = [x for x in food_lst if x != ' ']
# print(food_lst)

# Since we cannot retrieve data between <br> and </br>, lets manually do that
food_lst = ['Macgregors Meat & Seafood', 'Atlantic Grocery Distributors', 'Centennial', 'Chicago58',
            'FKK Wholesale Cash & Carey', 'Flanagan Foodservice', 'GFS', 'The Butcher Shoppe', 
            'Burton Meats', 'Nossack Fine Meats', 'Independent Fish', 'Italfoods', 'Mercury Wholesale',
            'Morton Wholesale', 'Northern Meat Services', 'Stewarts', 'Sysco', 'Vending Products of Canada',
            'Wallace & Carrey Inc.', 'Worldwide Specialty Foods']


In [54]:
address_food =['265 Garyray Dr', 'Bay Roberts', 'Richmond Hill', 'Woodbridge', 'North York',
          'Kitchener, Whitby, Walden Business Park Lively', 'Milton, Ajax', 'Etobicoke', 'Mississauga',
          'Red Deer', 'Winnipeg', 'Carp', 'Hamilton', 'Windsor', 'Winnipeg',
          'Barrie', 'Peterborough', 'Etobicoke', 'Oakville', 'Calgary']

In [55]:
lat_lng_list = []
not_avail = [] # There are several addresses that are not available in geopy geolocator

geolocator = Nominatim(user_agent='myGeocoder')
for distributor in address_food:
    city ="Toronto"
    country ="Canada"
    distributors = distributor.split(", ")
    if len(distributors) > 1:
        for i in range(len(distributors)):
            lat_lng_list.append([])
            location = geolocator.geocode(distributors[i]+','+city+','+ country)
            try:
                lat_lng_list[-1].append(location.latitude)
                lat_lng_list[-1].append(location.longitude)
                print('{}: Latitude = {}, Longitude = {}'.format(distributors[i], location.latitude, location.longitude))
            except:
                not_avail.append(distributors[i])
    else:
        lat_lng_list.append([])
        location = geolocator.geocode(distributor+','+city+','+ country)
        try:
            lat_lng_list[-1].append(location.latitude)
            lat_lng_list[-1].append(location.longitude)
            print('{}: Latitude = {}, Longitude = {}'.format(distributor, location.latitude, location.longitude))
        except:
            not_avail.append(distributor)

lat_lng_list = [lat_lng_list[x] for x in range(len(lat_lng_list)) if len(lat_lng_list[x]) != 0]          
# print(lat_lng_list)
# print(not_avail)


265 Garyray Dr: Latitude = 43.7689081, Longitude = -79.5395707
Bay Roberts: Latitude = 43.7359883, Longitude = -79.2589632
Richmond Hill: Latitude = 43.8125891, Longitude = -79.2633703893962
Woodbridge: Latitude = 43.7595855, Longitude = -79.59807320206491
North York: Latitude = 43.7543263, Longitude = -79.44911696639593
Kitchener: Latitude = 43.7319486, Longitude = -79.25356356130644
Milton: Latitude = 43.6953022, Longitude = -79.3320801
Etobicoke: Latitude = 43.6435559, Longitude = -79.5656326
Mississauga: Latitude = 43.6668555, Longitude = -79.5879563
Red Deer: Latitude = 43.6917658, Longitude = -79.2740401
Winnipeg: Latitude = 43.669121000000004, Longitude = -79.37185973720707
Hamilton: Latitude = 43.6706177, Longitude = -79.37468170845808
Windsor: Latitude = 43.66463645, Longitude = -79.37806090831586
Winnipeg: Latitude = 43.669121000000004, Longitude = -79.37185973720707
Barrie: Latitude = 43.6811352, Longitude = -79.4322816
Peterborough: Latitude = 43.6747137, Longitude = -79.44

In [56]:
market_df = pd.DataFrame({'Food Distributors':food_lst,
                         'Address':address_food})
# expand adresses of food distributor that has several locations and drop addresses that are in not_avail list
food_lst_mod = []
address_food_mod = []
for fd, s, n in zip(market_df['Food Distributors'], market_df['Address'], range(len(market_df['Address']))):
    if len(s) > 1:
        s_lst = s.split(", ")
        for i in range(len(s_lst)):
            if s_lst[i] not in not_avail:
                address_food_mod.append(s_lst[i])
                food_lst_mod.append(fd)
    else:
        food_lst_mod.append(fd)
        address_food_mod.append(s)
        
market_df = pd.DataFrame({'Food Distributors':food_lst_mod,
                         'Address':address_food_mod})
market_df

# create new DF for lat lng for each neighborhood
lat_list = [lat_lng_list[x][0] for x in range(len(lat_lng_list))]
lng_list = [lat_lng_list[x][1] for x in range(len(lat_lng_list))]
market_df = pd.DataFrame({'Food Distributors':market_df['Food Distributors'],
                          'Address':market_df['Address'],
                           'Latitude':lat_list,
                          'Longitude':lng_list})
market_df

,Food Distributors,Address,Latitude,Longitude
0,Macgregors Meat & Seafood,265 Garyray Dr,43.768908,-79.539571
1,Atlantic Grocery Distributors,Bay Roberts,43.735988,-79.258963
2,Centennial,Richmond Hill,43.812589,-79.263370
3,Chicago58,Woodbridge,43.759586,-79.598073
4,FKK Wholesale Cash & Carey,North York,43.754326,-79.449117
5,Flanagan Foodservice,Kitchener,43.731949,-79.253564
6,GFS,Milton,43.695302,-79.332080
7,The Butcher Shoppe,Etobicoke,43.643556,-79.565633
8,Burton Meats,Mississauga,43.666855,-79.587956
9,Nossack Fine Meats,Red Deer,43.691766,-79.274040


### Now that we found lat and lng for each food distributors, let's calculate the distance between our location candidates(St. James Town, Bay Street Corridor, North York City Centre)

In [57]:
# recall char_high_den_df
char_high_den_df.head(3)
print(char_high_den_df['Neighborhood'][:3])

0            St. James Town
1       Bay Street Corridor
2    North York City Centre
Name: Neighborhood, dtype: object


In [58]:
# get lat and lng for our candidates
neigh_lat_lng = []
geolocator = Nominatim(user_agent='myGeocoder')
for neighborhood in char_high_den_df['Neighborhood'][:3]:
    neigh_lat_lng.append([])
    city ="Toronto"
    country ="Canada"
    location = geolocator.geocode(neighborhood+','+city+','+ country)
    neigh_lat_lng[-1].append(location.latitude)
    neigh_lat_lng[-1].append(location.longitude)
print(neigh_lat_lng)

[[43.6694032, -79.3727041], [43.6673421, -79.3884571], [43.7543263, -79.44911696639593]]


In [59]:
lat_list = [neigh_lat_lng[x][0] for x in range(len(neigh_lat_lng))]
lng_list = [neigh_lat_lng[x][1] for x in range(len(neigh_lat_lng))]
top3_df = pd.DataFrame({'Neighborhood':char_high_den_df['Neighborhood'][:3],
                       'Latitude':lat_list,
                       'Longitude':lng_list})
top3_df

,Neighborhood,Latitude,Longitude
0,St. James Town,43.669403,-79.372704
1,Bay Street Corridor,43.667342,-79.388457
2,North York City Centre,43.754326,-79.449117


In [60]:
# lat and lng distance calculator (Haversine Formula)
import math
def dist_calculator(lat1, lng1, lat2, lng2):
    radius = 6371
    toRadian = math.pi / 180
    delta_Lat = abs(lat1 - lat2) * toRadian
    delta_Lng = abs(lng1 - lng2) * toRadian
    
    sin_Lat = math.sin(delta_Lat / 2)
    sin_Lng = math.sin(delta_Lng / 2)
    
    squareRoot = math.sqrt(
        sin_Lat * delta_Lat +
        math.cos(lat1 * toRadian) * math.cos(lat2 * toRadian) * sin_Lng * sin_Lng)
    
    distance = 2 * radius * math.asin(squareRoot)
    
    return distance

#### Distance between St.James Town and food distributors

In [61]:
dist_lst = []
for i in range(len(market_df['Food Distributors'])):
    dist_lst.append([])
    dist = dist_calculator(top3_df.loc[0, 'Latitude'], top3_df.loc[0, 'Longitude'], 
                            market_df.loc[i, 'Latitude'], market_df.loc[i, 'Longitude'])
    dist_lst[-1].append(market_df.loc[i, 'Food Distributors'])
    dist_lst[-1].append(round(dist, 2))
print(dist_lst)

[['Macgregors Meat & Seafood', 20.61], ['Atlantic Grocery Distributors', 13.9], ['Centennial', 24.17], ['Chicago58', 23.0], ['FKK Wholesale Cash & Carey', 14.7], ['Flanagan Foodservice', 13.73], ['GFS', 5.22], ['The Butcher Shoppe', 16.04], ['Burton Meats', 17.32], ['Nossack Fine Meats', 8.68], ['Independent Fish', 0.08], ['Mercury Wholesale', 0.25], ['Morton Wholesale', 0.86], ['Northern Meat Services', 0.08], ['Stewarts', 5.13], ['Sysco', 5.62], ['Vending Products of Canada', 16.04], ['Worldwide Specialty Foods', 0.13]]


In [62]:
food_markets = [x[0] for x in dist_lst]
dist_lst_mod = [x[1] for x in dist_lst]
stJames_dist_df = pd.DataFrame({'Food Distributors':food_markets,
                               'Distance to St.James (km)':dist_lst_mod})
stJames_dist_df.head()

,Food Distributors,Distance to St.James (km)
0,Macgregors Meat & Seafood,20.61
1,Atlantic Grocery Distributors,13.90
2,Centennial,24.17
3,Chicago58,23.00
4,FKK Wholesale Cash & Carey,14.70


In [63]:
print('The Mean Distance to St.James Town is {}'
      .format(stJames_dist_df['Distance to St.James (km)'].mean()))
stJames_dist_df.describe()

The Mean Distance to St.James Town is 10.308888888888891


,Distance to St.James (km)
count,18.000000
mean,10.308889
std,8.397219
min,0.080000
25%,1.927500
50%,11.205000
75%,16.040000
max,24.170000


#### Distance between Bay Street Corridor Town and food distributors

In [64]:
dist_lst1 = []
for i in range(len(market_df['Food Distributors'])):
    dist_lst1.append([])
    dist1 = dist_calculator(top3_df.loc[1, 'Latitude'], top3_df.loc[1, 'Longitude'], 
                            market_df.loc[i, 'Latitude'], market_df.loc[i, 'Longitude'])
    dist_lst1[-1].append(market_df.loc[i, 'Food Distributors'])
    dist_lst1[-1].append(round(dist1, 2))
print(dist_lst1)

[['Macgregors Meat & Seafood', 20.06], ['Atlantic Grocery Distributors', 15.0], ['Centennial', 24.95], ['Chicago58', 22.23], ['FKK Wholesale Cash & Carey', 14.52], ['Flanagan Foodservice', 14.86], ['GFS', 6.32], ['The Butcher Shoppe', 14.74], ['Burton Meats', 16.05], ['Nossack Fine Meats', 9.97], ['Independent Fish', 1.36], ['Mercury Wholesale', 1.22], ['Morton Wholesale', 0.94], ['Northern Meat Services', 1.36], ['Stewarts', 4.14], ['Sysco', 4.45], ['Vending Products of Canada', 14.74], ['Worldwide Specialty Foods', 1.18]]


In [65]:
food_markets1 = [x[0] for x in dist_lst1]
dist_lst_mod1 = [x[1] for x in dist_lst1]
bayStreet_dist_df = pd.DataFrame({'Food Distributors':food_markets1,
                               'Distance to Bay Street (km)':dist_lst_mod1})
bayStreet_dist_df.head()

,Food Distributors,Distance to Bay Street (km)
0,Macgregors Meat & Seafood,20.06
1,Atlantic Grocery Distributors,15.00
2,Centennial,24.95
3,Chicago58,22.23
4,FKK Wholesale Cash & Carey,14.52


In [66]:
print('The Mean Distance to Bay Street Corridor Town is {}'
      .format(bayStreet_dist_df['Distance to Bay Street (km)'].mean()))
bayStreet_dist_df.describe()

The Mean Distance to Bay Street Corridor Town is 10.449444444444445


,Distance to Bay Street (km)
count,18.000000
mean,10.449444
std,7.993305
min,0.940000
25%,2.055000
50%,12.245000
75%,14.965000
max,24.950000


#### Distance between North York City Centre and food distributors

In [67]:
dist_lst2 = []
for i in range(len(market_df['Food Distributors'])):
    dist_lst2.append([])
    dist2 = dist_calculator(top3_df.loc[2, 'Latitude'], top3_df.loc[2, 'Longitude'], 
                            market_df.loc[i, 'Latitude'], market_df.loc[i, 'Longitude'])
    dist_lst2[-1].append(market_df.loc[i, 'Food Distributors'])
    dist_lst2[-1].append(round(dist2, 2))
print(dist_lst2)

[['Macgregors Meat & Seafood', 7.62], ['Atlantic Grocery Distributors', 15.54], ['Centennial', 17.5], ['Chicago58', 11.99], ['FKK Wholesale Cash & Carey', 0.0], ['Flanagan Foodservice', 16.1], ['GFS', 13.21], ['The Butcher Shoppe', 19.78], ['Burton Meats', 17.71], ['Nossack Fine Meats', 17.17], ['Independent Fish', 14.77], ['Mercury Wholesale', 14.46], ['Morton Wholesale', 15.22], ['Northern Meat Services', 14.77], ['Stewarts', 11.59], ['Sysco', 12.53], ['Vending Products of Canada', 19.78], ['Worldwide Specialty Foods', 14.63]]


In [68]:
food_markets2 = [x[0] for x in dist_lst2]
dist_lst_mod2 = [x[1] for x in dist_lst2]
north_York_dist_df = pd.DataFrame({'Food Distributors':food_markets2,
                               'Distance to North York (km)':dist_lst_mod2})
north_York_dist_df.head()

,Food Distributors,Distance to North York (km)
0,Macgregors Meat & Seafood,7.62
1,Atlantic Grocery Distributors,15.54
2,Centennial,17.50
3,Chicago58,11.99
4,FKK Wholesale Cash & Carey,0.00


In [69]:
print('The Mean Distance to North York City Centre is {}'
      .format(north_York_dist_df['Distance to North York (km)'].mean()))
north_York_dist_df.describe()

The Mean Distance to North York City Centre is 14.131666666666668


,Distance to North York (km)
count,18.000000
mean,14.131667
std,4.614362
min,0.000000
25%,12.700000
50%,14.770000
75%,16.902500
max,19.780000


### Looking at the means of each, we can conclude that St.James Town and Bay Street Corridor are locared closer to food distributors in Ontario than North York Centre. 

### Now let's see the venues near St.James Town

In [70]:
a = neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Neighborhood']== 'St. James Town']
a

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
125,1,St. James Town,Coffee Shop,Pizza Place,Gym / Fitness Center,Grocery Store,Café,Hotel,Bar,Beer Store,Bistro,Breakfast Spot


### Now let's see the venues near Bay Street Corridor

In [71]:
b = neighborhoods_venues_sorted.loc[neighborhoods_venues_sorted['Neighborhood']== 'Bay Street Corridor']
b

,Cluster Labels,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,1,Bay Street Corridor,Clothing Store,Coffee Shop,Sushi Restaurant,French Restaurant,Boutique,Italian Restaurant,Japanese Restaurant,Mediterranean Restaurant,Café,Smoke Shop


### When we compare these two, we can notice that there are a lot more restaurants in Bay Street than St.James Town. Although it is almost 10-minutes drive distance, Bay street seems more suitable fo Ramsey's new italian restaurant than St.James Town. 

## Bay Street Corridor is our choice for Ramsey's new italian restaurant!

### Let's find his restaurant's possible competitors

In [272]:
CLIENT_ID = 'HT110NTAISUECGEVJNJ0ER0XPBTJGEUN5RTHQZX5TBI2JHHZ' # your Foursquare ID
CLIENT_SECRET = '5R332C1DX3M5FHJEPJHRVGMA1A41FJ01FJY13450I4OBAHKO' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HT110NTAISUECGEVJNJ0ER0XPBTJGEUN5RTHQZX5TBI2JHHZ
CLIENT_SECRET:5R332C1DX3M5FHJEPJHRVGMA1A41FJ01FJY13450I4OBAHKO


In [307]:
def getRestaurants (name, lat, lng, rad=500, lim=100):
    restaurant_list=[]   
    # request data from Foursquare
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId=4bf58dd8d48988d110941735'.format(
    CLIENT_ID, 
    CLIENT_SECRET,
    VERSION,
    lat, 
    lng, 
    rad, 
    lim)
        
    # make the GET request
    results = requests.get(url)#.json()["response"]['groups'][0]['items']
    results = results.json()["response"]
    try:
        results = results['groups'][0]['items']
    except:
        pass
        
    # return only relevant information for each nearby venue
    restaurant_list.append([(name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], 
                         v['venue']['location']['lng'],  v['venue']['categories'][0]['name']) for v in results])

    nearby_restaurant = pd.DataFrame([item for restaurant_list in restaurant_list for item in restaurant_list])
    nearby_restaurant.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_restaurant)


In [309]:
res_df = getRestaurants('Bay Street Corridor', top3_df.loc[1, 'Latitude'], top3_df.loc[1, 'Longitude'])
res_df

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Street Corridor,43.667342,-79.388457,Trattoria Nervosa,43.671019,-79.391081,Italian Restaurant
1,Bay Street Corridor,43.667342,-79.388457,Eataly,43.669754,-79.388720,Gourmet Shop
2,Bay Street Corridor,43.667342,-79.388457,Dal Moro's Fresh Pasta To Go,43.666641,-79.385400,Italian Restaurant
3,Bay Street Corridor,43.667342,-79.388457,7 West Cafe,43.668665,-79.386830,Italian Restaurant
4,Bay Street Corridor,43.667342,-79.388457,Blu Ristorante and Lounge,43.671685,-79.388614,Italian Restaurant
5,Bay Street Corridor,43.667342,-79.388457,Dimmi,43.670062,-79.392680,Italian Restaurant
6,Bay Street Corridor,43.667342,-79.388457,Tutti Pizzeria,43.670121,-79.391258,Pizza Place
7,Bay Street Corridor,43.667342,-79.388457,Vaticano Trattoria,43.670738,-79.391043,Italian Restaurant
8,Bay Street Corridor,43.667342,-79.388457,Scaccia Restaurant,43.668794,-79.388296,Italian Restaurant
9,Bay Street Corridor,43.667342,-79.388457,Sofia,43.670592,-79.391849,Italian Restaurant


In [323]:
res_df_mod = res_df[res_df['Venue Category'] == 'Italian Restaurant']
res_df_mod.drop(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude'], axis=1)

,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Trattoria Nervosa,43.671019,-79.391081,Italian Restaurant
2,Dal Moro's Fresh Pasta To Go,43.666641,-79.385400,Italian Restaurant
3,7 West Cafe,43.668665,-79.386830,Italian Restaurant
4,Blu Ristorante and Lounge,43.671685,-79.388614,Italian Restaurant
5,Dimmi,43.670062,-79.392680,Italian Restaurant
7,Vaticano Trattoria,43.670738,-79.391043,Italian Restaurant
8,Scaccia Restaurant,43.668794,-79.388296,Italian Restaurant
9,Sofia,43.670592,-79.391849,Italian Restaurant
10,Focaccia,43.669932,-79.386110,Italian Restaurant
11,Zaza paninoteca,43.670739,-79.391159,Italian Restaurant


### Now let's narrow it down by price range of the restaurants
* <b>Trattoria Nervosa</b> ---> \$\$
* <b>Dal Moro's Fresh Pasta To Go</b> ---> \$
* <b>7 West Cafe</b> ---> \$\$
* <b>Blu Ristorante and Lounge</b> ---> \$\$\$
* <b>Dimmi</b> ---> \$\$
* <b>Vaticano Trattoria</b> ---> \$\$
* <b>Scaccia Restaurant</b> ---> \$
* <b>Sofia</b> ---> (4 \$s)
* <b>Focaccia</b> ---> \$
* <b>Zaza paninoteca</b> ---> \$\$
* <b>Bacaro Italian Eatery</b> ---> closed

### Who is the possible competitor?
#### Considering Ramsay's restaurants are luxurious (probably between 3 to 5 dollar signs), the possible competitors against his restaurant would be:
* <b>Blu Ristorante and Lounge</b>
* <b>Sofia</b>

## Result

Under the assumption that Gordon Ramsey is looking for a location for his new italian restaurant in Toronto, Bay Street Corridor would be the best choice for his new italian restaurant. Bay Street Corridor neighborhood has the highest floating population in Toronto except St.James Town where its high population is highly concentrated on residents only. In addition, Ramsey's restaurant would have the upper hand compared to other restaurants because Bay Street Corridor is located near variety of food distributors, giving Ramsey's restaurant the ability to get fresh ingredients. When we think about his restaurant's possible competitors, there are Blu Ristorante and Lounge and Sofia. 

## Discussion

Toronto is a big city with diverse culture and population density. My approach to the business problem was simple: I tried to focus on the floating population of each neighborhoods in Toronto. With K-cluster of 5, I utilized K-Means Clustering method in order to divide the data into separate clusters. I then used Folium to map the result out. 

I then found the characteristics of each of the neighborhoods which have high floating population. I could exclude the nieghborhods that does not fit to Ramsey's italian restaurant. The final 3 neighborhoods were St. James Town, Bay Street Corridor, and North York City Centre.

Since all three neighborhoods are located in Ontario, Toronto, I then focused on the distance between the neighborhoods and food distributors in Ontrio. I utilized BeautifulSoup in order to read data of food distributors in http://betterfoodconcepts.ca/distributors/. I also utilized geocoder in order to find the latitude and longitude for each of the food distributors. I then used Haversine Formula for distance between each of the neighborhoods and the food distributors. From its results, I could rule out North City Centre which has longest mean distance of 14.13 km. 

Through Foursquare API, I explored venues near St.James Town and Bay Street Corridor in radius of 500. As I expected through the neighborhoods' characteristics, the most commmon venues near St.James Town were cafe, gyms, and beer shops as St.James Town is the resident-centered neighborhood. On the other hand, the most commmon venues near Bay Street Corridor were restaurants and clothing shops. From this data analysis, I could derive insights that Bay Street Corrider is the best neighborhood for Ramsey's new itatlian restaurant.

Lastly, I found possible competitors against Ramsey's restaurant. Also using Foursquare API, I could gather data on italian restaurants in Bay Street Corridor. Among 11 italian restaurants in the neighborhood, I ruled out the restaurants that are as not luxurious as most of Ramsey's restaurant. The possible competitors against Ramsey's restaurant were Blu Ristorante and Lounge and Sofia.

## Conclusion

In conclusion, Bay Street Corridor is the best neighborhood for Ramsey's new italian restaurant when we take account of floating population, characteristics of the neighborhood, and distance between the neighborhood and food distributors.

Opening an italian restaurant in Bay Street Corridor will not only expose the restaurant to many people but also maintain Ramsey's reputation for Michellen-star chef with quick distribution of fresh ingredients. 